In [258]:
batch_size = 4
block_size = 512
max_iters = 200
learning_rate = 2e-5
eval_iters = 100
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.2

cuda


In [259]:
# Load the BERT uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Get the vocabulary size
vocab_size = len(tokenizer)

In [260]:
# Encode and decode functions using the tokenizer
encode = lambda s: tokenizer.encode(s, add_special_tokens=True)
decode = lambda l: tokenizer.decode(l, skip_special_tokens=True)

In [261]:
def load_half_dataset_into_memory(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        f.seek(0, 2)  # Move the cursor to the end of the file
        half_point = f.tell() // 200  # Find the halfway point
        f.seek(0)  # Reset cursor to the beginning
        data = f.read(half_point)  # Read up to the halfway point
    
    return data


# Preprocess and encode your dataset (1/2 of it), then convert it to tensor
train_data = load_half_dataset_into_memory("output_train.txt")
val_data = load_half_dataset_into_memory("output_val.txt")

# Assuming you have a function encode() that converts text to a list of integers
train_encoded = torch.tensor(encode(train_data), dtype=torch.long)
val_encoded = torch.tensor(encode(val_data), dtype=torch.long)

Token indices sequence length is longer than the specified maximum sequence length for this model (406817 > 512). Running this sequence through the model will result in indexing errors


In [263]:
def get_batch(split):
    # start_time = time.time()
    # Select the appropriate dataset based on the split
    data = train_encoded if split == 'train' else val_encoded
    
    # Ensure we have enough data to sample from
    if data.size(0) > block_size:
        ix = torch.randint(0, data.size(0) - block_size, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    else:
        raise ValueError("Dataset size is too small for the requested block and batch sizes.")
    
    # Assuming 'device' is defined (e.g., 'cuda' or 'cpu')
    x, y = x.to(device), y.to(device)
    # print("--- %s seconds ---" % (time.time() - start_time))
    return x, y

In [264]:
model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [266]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # Ensure we have enough data to sample from
    if train_encoded.size(0) > block_size:
        ix = torch.randint(0, train_encoded.size(0) - block_size, (batch_size,))
        x = torch.stack([train_encoded[i:i+block_size] for i in ix])
        y = torch.stack([train_encoded[i+1:i+block_size+1] for i in ix])
    else:
        raise ValueError("Dataset size is too small for the requested block and batch sizes.")
    
    # evaluate the loss
    logits, loss = model.forward(x.to(device), y.to(device))
    # logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
torch.Size([4, 512, 4, 256])


RuntimeError: The size of tensor a (512) must match the size of tensor b (4) at non-singleton dimension 2

In [ ]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)